<a href="https://colab.research.google.com/github/Mong-Gu/DataScience/blob/master/part1_Lab02_Simple_Linear_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf

#**가설함수 H(x) = Wx + b**

In [0]:
x_data = [1, 2, 3, 4, 5]
y_data = [1, 2, 3, 4, 5]

W = tf.Variable(2.9) # 기울기
b = tf.Variable(0.5) # 절편

hypothesis = W * x_data + b

x_data 와 y_data가 위와 같은 상태라면, 최적의 W = 1, b = 0이다. 이후 W와 b가 조정될 것인지 확인해보자.
#### <참고> 일반적으로 W와 b의 초기값은 랜덤값으로 구현한다

---

# **비용함수 cost(W, b)**

In [0]:
cost = tf.reduce_mean(tf.square(hypothesis - y_data))

오차의 제곱의 합을 평균, 즉 MSE(Mean Squared Error) 기반의 비용함수이다.

* tf_reduce_mean() : 차원(랭크)이 줄어들면서 평균을 구하는 함수

  ex)<br>
  v = [1., 2., 3., 4.]<br>
  tf.reduce_mean(v) # 2.5<br>
  이처럼 1차원에서 0차원으로 줄어들게 된다

  <br>

* tf.square() : parameter를 제곱해주는 함수

---

# **경사하강법 (Gradient Descent)**

비용을 최소화하는 알고리즘은 많은데, 그중 가장 흔히 쓰이는 것이 경사하강법이다.

In [0]:
# Learning_rate initialize
learning_rate = 0.01

In [6]:
# Gradient Descent
with tf.GradientTape() as tape:
  hypothesis = W * x_data + b
  cost = tf.reduce_mean(tf.square(hypothesis - y_data))

W_grad, b_grad = tape.gradient(cost, [W, b])

W.assign_sub(learning_rate * W_grad)
b.assign_sub(learning_rate * b_grad)

<tf.Variable 'UnreadVariable' shape=() dtype=float32, numpy=0.376>

tensorflow에서는 Gradient Descent를 일반적으로 with구문과 함께 GradientTape()으로 구현한다.

<순서>
1. (line 2~4)
with 구문 안에 있는 변수 정보를 tape에 기록한다. 위 예에서는 W와 b가 담길 것이다.
2. (line 6)
이후 tape에 gradient method를 호출하여 경사도값(기울기), 즉 미분값을 구한다.
  - gradient method는 cost 함수에 대해서 변수들 (W, b)에 대한 개별 미분값을 구하여 튜플로 반환한다.
  - 반환된 값들은 순서대로 W_grad과 b_grad에 담긴다. 즉, W에 대한 미분값은 W_grad에 순서대로 튜플로 저장되고, b에 대한 미분값은 b_grad에 순서대로 튜플로 저장된다.
3. (line 8, 9)
W와 b를 업데이트해준다.
  - 여기서 사용된 A.assign_sub(B)는 python에서 'A = A - B' 혹은 ' A -= B'와 동일한 연산을 수행한다. 
    - 다만 tensorflow 내에서는 python 문법을 활용할 수 없기 때문에 assign_sub를 활용한다.
  - 각 gradient값에 learning_rate 값을 곱해서 업데이트해주고 있는데, learning_rate는 gradient값을 얼마만큼 반영해줄 것인지 정해준다.
    - 일반적으로 learning_rate는 굉장히 작은 값으로 할당한다. 0.01, 0.001, 0.0001, .... learning_rate를 너무 많이 할당해버리면 빠르게 학습을 할 수 있겠지만, 정확도가 떨어질 위험이 있다. 조금씩 차근차근 반영해나가며 최적의 W와 b를 찾아나가는 것이 더 좋다.


**위 과정이 W와 b를 업데이트하는 한 걸음이라고 생각하자.**

---

# **위 과정을 모두 반영하여 총정리한 Full Code**

In [10]:
import tensorflow as tf
# tf.enable_eager_execution()

# Data
x_data = [1, 2, 3, 4, 5]
y_data = [1, 2, 3, 4, 5]

# W, b initialize
W = tf.Variable(2.9)
b = tf.Variable(0.5)

learning_rate = 0.01

# Update both W and b
for i in range(100+1):
  # Gradient Descent
  with tf.GradientTape() as tape:
    hypothesis = W * x_data + b
    cost = tf.reduce_mean(tf.square(hypothesis - y_data))
  
  W_grad, b_grad = tape.gradient(cost, [W, b])
  W.assign_sub(learning_rate * W_grad)
  b.assign_sub(learning_rate * b_grad)

  if i % 10 == 0:
    print("{:5}|{:10.4f}|{:10.4}|{:10.6f}".format(i, W.numpy(), b.numpy(), cost))

    0|    2.4520|     0.376| 45.660004
   10|    1.1036|  0.003398|  0.206336
   20|    1.0128|  -0.02091|  0.001026
   30|    1.0065|  -0.02184|  0.000093
   40|    1.0059|  -0.02123|  0.000083
   50|    1.0057|  -0.02053|  0.000077
   60|    1.0055|  -0.01984|  0.000072
   70|    1.0053|  -0.01918|  0.000067
   80|    1.0051|  -0.01854|  0.000063
   90|    1.0050|  -0.01793|  0.000059
  100|    1.0048|  -0.01733|  0.000055


i = 0 (반복 횟수; Epoch = 1)일 때는 W가 2.4520, b가 0.3760인 반면, 학습이 이루어질수록 앞서 생각했던 W = 1, b = 0으로 수렴하고 있다. 즉, 점점 cost가 줄어들고 있다는 것이다.

---

그렇다면 만들어놓은 가설 함수는 최종적으로 다음과 같은 모양을 가지게 된다.<br>
**H(x) = 1.0048 * x - 0.01733**

이 가설 함수로 새로운 값을 예측해보자,

In [12]:
print(W * 5 + b)
# 기존의 x_data에서는 x = 5일 때 y = 5였다. 
# y가 5에 가까울수록 정확한 가설 함수를 세웠다고 할 수 있다.

tf.Tensor(5.00667, shape=(), dtype=float32)


In [13]:
print(W * 2.5 + b)
# 기존의 x_data와 y_data를 살펴봤을 때, 
# y는 2.5에 가까울수록 정확한 가설 함수를 세웠다고 할 수 있다.

tf.Tensor(2.4946702, shape=(), dtype=float32)
